# Techno-Economic Feasibility Analysis of Small-Scale PV-Battery System in Australia

Description: This project aims to assess the techno-economic feasibility of a small-scale Photovoltaic (PV)-battery system. It investigates factors like electricity pricing, solar irradiance, and battery storage costs to evaluate the system's ROI and sustainability.

## Objective

The primary focus of this project is to develop and optimise a Home Energy Management System (HEMS) aimed at minimising electricity costs for households equipped with rooftop solar and a battery storage system. The objective function aims to determine the optimal times for charging and discharging the battery, subject to equipment constraints and power balance requirements. The specific objectives are:

* To model a basic HEMS that allows for control over rooftop solar and battery storage.
* To minimise the overall electricity expenditure by leveraging solar energy and battery storage.
* To find the optimal times for battery charging and discharging, subject to constraints like no simultaneous charging and discharging.
* To perform this analysis at a half-hourly time resolution, considering both energy costs and tariffs for energy sent back to the grid.


---

## Set up


In [35]:
!pip install python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [36]:
import pandas as pd
import numpy as np
import os
import pathlib

from amplpy import AMPL, DataFrame, tools
from dotenv import load_dotenv


In [37]:
load_dotenv(pathlib.Path.cwd() / '.env')

ampl = tools.ampl_notebook(
    modules=["cplex"],
    license_uuid=os.environ.get('AMPL_UUID'),
)

Licensed to Bundle #6115.6438 expiring 20240131: ELEC5213 Engineering Optimisation, Professor Gregor Verbic, The University of Sydney.


---

## Pre-process Data

 - Handled negative PV generation values by setting them to zero.
 - Converted total_load from kW to kWh, assuming each time step is 30 minutes.
 - Converted pv_generation from kW to kWh, assuming each time step is 30 minutes.
 - Calculated net_load as total_load - pv_generation.

In [38]:
site_id = 152786204
df = pd.read_csv(f'data/HalfHourly_PV_Load_Data/halfhourly_{site_id}.csv')
assert df['site_id'][0] == site_id

df['utc_timestamp'] = pd.to_datetime(df['utc_timestamp'])
df['localtime'] = pd.to_datetime(df['localtime'])

df.set_index('utc_timestamp', inplace=True)

df['pv_generation'] = df['pv_generation'].apply(lambda x: max(0, x)) # set negative values to zero

df['total_load_kWh'] = df['total_load'] * 0.5  # for 30 minute intervals
df['pv_generation_kWh'] = df['pv_generation'] * 0.5  # for 30 minute intervals

df['net_load_kWh'] = df['total_load_kWh'] - df['pv_generation_kWh']

print(df.head(3))


                             site_id  pv_generation  total_load  \
utc_timestamp                                                     
2018-12-31 13:00:00+00:00  152786204            0.0   28.504320   
2018-12-31 13:30:00+00:00  152786204            0.0   27.113783   
2018-12-31 14:00:00+00:00  152786204            0.0   21.089683   

                                           localtime  total_load_kWh  \
utc_timestamp                                                          
2018-12-31 13:00:00+00:00  2019-01-01 00:00:00+11:00       14.252160   
2018-12-31 13:30:00+00:00  2019-01-01 00:30:00+11:00       13.556892   
2018-12-31 14:00:00+00:00  2019-01-01 01:00:00+11:00       10.544842   

                           pv_generation_kWh  net_load_kWh  
utc_timestamp                                               
2018-12-31 13:00:00+00:00                0.0     14.252160  
2018-12-31 13:30:00+00:00                0.0     13.556892  
2018-12-31 14:00:00+00:00                0.0     10.544842 

---

## Set parameters



In [39]:
slots_per_day = 48
num_days = 1
num_time_slots = slots_per_day * num_days

# Tariffs in $/kWh
time_of_use_tariff = 0.27  # c_g - constant value for now, but replace with csv data from Ausgrid
flat_tariff = 0.25  # c_flat
feed_in_tariff = 0.05  # c_pv

demand_data = df['total_load_kWh'][:num_time_slots].values  # Pd
pv_data = df['pv_generation_kWh'][:num_time_slots].values  # Ppv

battery_max_storage = 10.0  # ebM, in kWh
battery_min_storage = 0.0
battery_max_charge_rate = 5.0  # PbM, in kW
battery_max_discharge_rate = 5.0  # Pbm, in kW

battery_charge_efficiency = np.sqrt(0.84) # etaBc
battery_discharge_efficiency = battery_charge_efficiency  # etaBd

---

## Model Formulation

### Indices

$i, j \in \text{Households})$: Index to represent individual households

$t \in \text{Time Periods})$: Index to represent different time periods

### Parameters

$c_t$: Cost for electricity from the grid at time *t*

$s_t$: Solar power generated at time *t*

$d_t$: Demand for electricity at household *i* at time *t*

$r_t$: Tariff for power sent back to the grid at time *t*

### Decision Variables

$p_t$: Power drawn from the grid at time *t*

$b_t$: Battery state (charging/discharging) at time $t$

### Objective Function

- **Cost Minimisation**: The objective is to minimise the total electricity cost for the household, while considering tariffs for power sent back to the grid.

\begin{equation}
\text{Minimize: } C(p, b) = \sum_{t} (p_{t} \times c_{t}) - (s_{t} - d_{t}) \times r_{t}
\end{equation}

### Constraints

1. **Power Balance**: The power drawn from the grid, plus solar, should meet the demand.

\begin{equation}
p_{t} + s_{t} = d_{t}
\end{equation}

2. **Battery Operations**: No simultaneous charging and discharging.

\begin{equation}
\text{If } b_{t} = \text{charging, then } b_{t} \neq \text{discharging}
\end{equation}

3. **Half-Hourly Resolution**: All variables and parameters are considered at a half-hourly time resolution.

\begin{equation}
\forall t \in \text{Half-hour intervals}
\end{equation}


In [40]:

# days_set = ampl.getSet('D')
# print(days_set)

# df = DataFrame(('D',), time_slots_two_days)

# Dummy data - replace these with actual data
# PV = np.zeros((DAILY_TIME_SLOTS_AMT*DAYS_IN_WEEK + 2*DAILY_TIME_SLOTS_AMT, CUSTOMERS_AMT))
# PD = np.zeros((DAILY_TIME_SLOTS_AMT*DAYS_IN_WEEK + 2*DAILY_TIME_SLOTS_AMT, CUSTOMERS_AMT))

KeyError: 'An entity called D cannot be found.'

In [ ]:
# PV Sizes for 52 customers in kW
pv_sizes = [5, 4, 5, 3, 6, 6, 5, 4, 4, 9, 3, 4, 8, 4, 6, 5, 10, 5, 4, 4, 5, 4, 7, 4, 4, 4, 5, 7, 7, 5, 5, 4, 5, 3, 5, 5, 5, 10, 5, 5, 7, 5, 5, 3, 6, 3, 4, 5, 5, 5, 4, 4]

# Battery stats for three types of EVs (soc = state of charge)
max_soc = [40, 40, 40]
min_soc = [0.6, 1.0, 0.5]
max_charge_discharge_rate = [4.2, 5.0, 5.0]  # Maximum charge/discharge rates
charge_discharge_efficiency = np.sqrt([0.9, 0.9, 0.9])  # Charge and discharge efficiencies
initial_soc = [x / 2 for x in max_soc]